In [23]:
import gym
import numpy as np

In [24]:
# 创建环境
env = gym.make('CliffWalking-v0',render_mode='human')

In [25]:
from torch import nn
import torch

In [26]:
class PolicyNet(nn.Module):
    
    def __init__(self,input_dim,output_dim):
        super().__init__()
        self.linear1 = nn.Linear(input_dim,200)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(200,output_dim)
        self.softmax = nn.Softmax(dim=-1)
    
    def forward(self,state):
        ### n
        x = self.linear1(state)
        x = self.relu(x)
        x = self.linear2(x) # n
        x = self.softmax(x) # n
        return x

In [27]:
class ValueNet(nn.Module):
    ### 用来学习值
    
    def __init__(self,input_dim):
        super().__init__()
        self.linear1 = nn.Linear(input_dim,200)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(200,1)
    
    def forward(self,state):
        ### n
        x = self.linear1(state)
        x = self.relu(x)
        x = self.linear2(x) # n
        return x

In [28]:
from torch.distributions import Categorical
import numpy as np
np.bool8 = np.bool_

from torch.optim import AdamW

In [29]:
class Agent:
    
    def __init__(self):
        self.policy_net = PolicyNet(48,4)
        self.value_net = ValueNet(48)
        self.optimizer = AdamW(self.policy_net.parameters(),lr=1e-3)
        self.value_optimizer = AdamW(self.value_net.parameters(),lr=1e-3)
    
    def sample_action(self,state):
        probs = self.policy_net(state) # 4
        if np.random.uniform() < 0.5:
            action = np.random.randint(0,4)
            return action, torch.log(probs[action]+1e-8).detach()
        dist = Categorical(probs)
        action = dist.sample()
        log_prob = dist.log_prob(action).detach()
        return action.item(),log_prob
    
    def update(self,rewards,log_probs,xs,old_actions):
        ### 一次游戏时间
        ret = []
        adding = 0
        for r in rewards[::-1]:
            adding = adding * 0.99 + r
            ret.insert(0,adding)
        ret = torch.FloatTensor(ret)
        ret = ret - ret.mean()
        ret = ret / (ret.std()+1e-8)
        
        
        
        for _ in range(4):
            values = self.value_net(xs) ## B,48 ==> B,1
#             print(values.shape)

            new_probs = self.policy_net(xs) ## B,4
            dist = Categorical(new_probs) # B,4
#             actions = dist.sample() # B,1
            new_logprobs = dist.log_prob(old_actions.squeeze(-1)) # B,1
#             print('new log probs',new_logprobs.shape)
            advantages = ret - values.squeeze(-1).detach() # B
#             print('adv shape',advantages.shape,'ret shape',ret.shape,'value shape',values.shape,'log',log_probs.shape)
#             print('new_probs',new_probs.shape,'new_logprobs shape',new_logprobs.shape,'old_actions',old_actions.shape)
            ratio = torch.exp(new_logprobs - log_probs.squeeze(-1)) ## B
#             print('shape',ratio.shape)
            surr1 = ratio * advantages.squeeze(-1)
            surr2 = torch.clamp(ratio,0.8,1.2) * advantages
            loss = -torch.min(surr1,surr2).mean()


            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()


            value_loss = (values.squeeze(0) - ret).pow(2).mean()
            self.value_optimizer.zero_grad()
            value_loss.backward()
            self.value_optimizer.step()


    #         r_log_probs = []
    #         for r,log_prob in zip(ret,log_probs):
    #             r_log_probs.append(-r*log_prob)
    #         r_log_probs = torch.vstack(r_log_probs)

    #         loss = r_log_probs.sum()



            return loss

In [30]:
def convert2tensor(state):
    state_arr = np.zeros(48)
    state_arr[state] = 1
    state_arr = torch.FloatTensor(state_arr)
    return state_arr

In [31]:
def train(agent,env):
    success_count = []
    max_size = 1000
    for epoch in range(200000):
        rewards = []
        xs = []
        log_probs = []
        old_actions = []
        terminated = False
        success = True
        state,_ = env.reset()
        while not terminated and len(log_probs) < max_size:
            state_arr = convert2tensor(state)
            xs.append(state_arr)
            action, log_prob = agent.sample_action(state_arr)
            next_state, reward, terminated, truncated, _ = env.step(action)
            if reward == -100:
                reward = -10
                success = False
            if reward == -1 and state == 47:
                reward = 10
            state = next_state
            rewards.append(reward)
            log_probs.append(log_prob)
            old_actions.append(action)
        xs = torch.vstack(xs)
        log_probs = torch.vstack(log_probs)
        old_actions = torch.LongTensor(old_actions)
        loss = agent.update(rewards,log_probs,xs,old_actions) 
        success_count.append(success)
        
        
        if (epoch+1) % 10 == 0:
            print(f'success rate:  {len([s for s in success_count[-100:] if s]) / 100}')
            print(f'epoch: {epoch}, loss: {loss}, rewards: {sum(rewards)}, count: {len(rewards)}')

In [32]:
agent = Agent()



In [33]:
env = gym.make('CliffWalking-v0')
train(agent,env)

success rate:  0.0
epoch: 9, loss: -0.0005002827383577824, rewards: -1489, count: 913
success rate:  0.0
epoch: 19, loss: 0.0009156869491562247, rewards: -1684, count: 1000
success rate:  0.0
epoch: 29, loss: -0.00020552253408823162, rewards: -1468, count: 1000
success rate:  0.0
epoch: 39, loss: -0.0006219861679710448, rewards: -1522, count: 1000
success rate:  0.0
epoch: 49, loss: 0.00031052465783432126, rewards: -307, count: 253
success rate:  0.01
epoch: 59, loss: 0.0002517204266041517, rewards: -1405, count: 1000
success rate:  0.01
epoch: 69, loss: -0.00013774204126093537, rewards: -1216, count: 1000
success rate:  0.01
epoch: 79, loss: -0.00012465739564504474, rewards: -1117, count: 1000
success rate:  0.01
epoch: 89, loss: -1.9902738131349906e-05, rewards: -460, count: 343
success rate:  0.02
epoch: 99, loss: 1.6390324162784964e-05, rewards: -1153, count: 1000
success rate:  0.03
epoch: 109, loss: 0.00012695646728388965, rewards: -1099, count: 1000
success rate:  0.03
epoch: 11

success rate:  0.33
epoch: 969, loss: 0.40849626064300537, rewards: -198, count: 171
success rate:  0.33
epoch: 979, loss: 0.4631671905517578, rewards: -74, count: 74
success rate:  0.36
epoch: 989, loss: 0.3255077302455902, rewards: -465, count: 411
success rate:  0.39
epoch: 999, loss: 0.6999964714050293, rewards: -53, count: 53
success rate:  0.35
epoch: 1009, loss: 0.42863205075263977, rewards: -510, count: 465
success rate:  0.34
epoch: 1019, loss: 0.47540390491485596, rewards: -234, count: 234
success rate:  0.3
epoch: 1029, loss: 0.48522308468818665, rewards: -216, count: 207
success rate:  0.33
epoch: 1039, loss: 0.4274773895740509, rewards: -306, count: 297
success rate:  0.33
epoch: 1049, loss: 0.8529460430145264, rewards: -88, count: 88
success rate:  0.32
epoch: 1059, loss: 0.7436782717704773, rewards: -54, count: 54
success rate:  0.33
epoch: 1069, loss: 0.4645916223526001, rewards: -194, count: 185
success rate:  0.32
epoch: 1079, loss: 0.5357823967933655, rewards: -104, 

success rate:  0.39
epoch: 1939, loss: 1.0798730850219727, rewards: -273, count: 246
success rate:  0.41
epoch: 1949, loss: 1.5742977857589722, rewards: -62, count: 62
success rate:  0.4
epoch: 1959, loss: 0.5436072945594788, rewards: -294, count: 267
success rate:  0.41
epoch: 1969, loss: 1.1786221265792847, rewards: -406, count: 397
success rate:  0.38
epoch: 1979, loss: 1.048628330230713, rewards: -637, count: 619
success rate:  0.39
epoch: 1989, loss: 1.207743763923645, rewards: -204, count: 204
success rate:  0.42
epoch: 1999, loss: 1.1155527830123901, rewards: -393, count: 366
success rate:  0.38
epoch: 2009, loss: 0.739505410194397, rewards: -217, count: 199
success rate:  0.36
epoch: 2019, loss: 1.2092152833938599, rewards: -262, count: 226
success rate:  0.38
epoch: 2029, loss: 1.1085785627365112, rewards: -53, count: 44
success rate:  0.38
epoch: 2039, loss: 1.3647011518478394, rewards: -224, count: 206
success rate:  0.35
epoch: 2049, loss: 1.20438814163208, rewards: -162, c

success rate:  0.31
epoch: 2919, loss: 0.4944266378879547, rewards: -318, count: 282
success rate:  0.31
epoch: 2929, loss: 1.7381693124771118, rewards: -213, count: 168
success rate:  0.38
epoch: 2939, loss: 0.4490331709384918, rewards: -93, count: 93
success rate:  0.42
epoch: 2949, loss: 0.7737452983856201, rewards: -59, count: 59
success rate:  0.46
epoch: 2959, loss: 1.6295057535171509, rewards: -35, count: 35
success rate:  0.47
epoch: 2969, loss: 0.8736204504966736, rewards: -61, count: 52
success rate:  0.43
epoch: 2979, loss: 0.5858422517776489, rewards: -471, count: 426
success rate:  0.41
epoch: 2989, loss: 1.221670389175415, rewards: -219, count: 210
success rate:  0.41
epoch: 2999, loss: 0.6726319193840027, rewards: -461, count: 407
success rate:  0.37
epoch: 3009, loss: 1.34487783908844, rewards: -79, count: 79
success rate:  0.38
epoch: 3019, loss: 1.2180659770965576, rewards: -203, count: 194
success rate:  0.39
epoch: 3029, loss: 0.7393692135810852, rewards: -279, coun

success rate:  0.38
epoch: 3899, loss: 0.8594244122505188, rewards: -80, count: 80
success rate:  0.35
epoch: 3909, loss: 1.3144080638885498, rewards: -47, count: 47
success rate:  0.36
epoch: 3919, loss: 0.6799717545509338, rewards: -146, count: 137
success rate:  0.41
epoch: 3929, loss: 1.0925215482711792, rewards: -455, count: 410
success rate:  0.42
epoch: 3939, loss: 1.0549952983856201, rewards: -124, count: 115
success rate:  0.39
epoch: 3949, loss: 1.3488167524337769, rewards: -78, count: 78
success rate:  0.34
epoch: 3959, loss: 1.2383522987365723, rewards: -103, count: 94
success rate:  0.34
epoch: 3969, loss: 0.7669990658760071, rewards: -200, count: 191
success rate:  0.36
epoch: 3979, loss: 0.922884464263916, rewards: -483, count: 411
success rate:  0.35
epoch: 3989, loss: 1.402268886566162, rewards: -206, count: 170
success rate:  0.34
epoch: 3999, loss: 1.6786166429519653, rewards: -161, count: 152
success rate:  0.32
epoch: 4009, loss: 0.39825090765953064, rewards: -180,

...

success rate:  0.4
epoch: 195499, loss: 1.1597262620925903, rewards: -402, count: 384
success rate:  0.39
epoch: 195509, loss: 1.15739905834198, rewards: -83, count: 83
success rate:  0.4
epoch: 195519, loss: 1.8667904138565063, rewards: -50, count: 50
success rate:  0.38
epoch: 195529, loss: 1.7602285146713257, rewards: -113, count: 113
success rate:  0.38
epoch: 195539, loss: 1.2760506868362427, rewards: -49, count: 49
success rate:  0.36
epoch: 195549, loss: 1.442826747894287, rewards: -165, count: 156
success rate:  0.35
epoch: 195559, loss: 0.8838554620742798, rewards: -141, count: 132
success rate:  0.32
epoch: 195569, loss: 1.3171907663345337, rewards: -206, count: 170
success rate:  0.32
epoch: 195579, loss: 1.3421698808670044, rewards: -112, count: 94
success rate:  0.35
epoch: 195589, loss: 0.7817897200584412, rewards: -52, count: 52
success rate:  0.34
epoch: 195599, loss: 1.2750749588012695, rewards: -120, count: 111
success rate:  0.34
epoch: 195609, loss: 1.25224828720092

success rate:  0.36
epoch: 196459, loss: 1.1784703731536865, rewards: -44, count: 35
success rate:  0.38
epoch: 196469, loss: 1.45524001121521, rewards: -137, count: 128
success rate:  0.41
epoch: 196479, loss: 1.3714134693145752, rewards: -236, count: 209
success rate:  0.43
epoch: 196489, loss: 1.2389320135116577, rewards: -403, count: 331
success rate:  0.41
epoch: 196499, loss: 1.6032052040100098, rewards: -107, count: 98
success rate:  0.44
epoch: 196509, loss: 1.296581745147705, rewards: -240, count: 222
success rate:  0.4
epoch: 196519, loss: 1.573933720588684, rewards: -207, count: 198
success rate:  0.37
epoch: 196529, loss: 1.7487331628799438, rewards: -60, count: 60
success rate:  0.38
epoch: 196539, loss: 1.310745358467102, rewards: -207, count: 189
success rate:  0.35
epoch: 196549, loss: 1.129062533378601, rewards: -54, count: 45
success rate:  0.37
epoch: 196559, loss: 1.273952841758728, rewards: -349, count: 322
success rate:  0.34
epoch: 196569, loss: 1.427671909332275

success rate:  0.39
epoch: 197419, loss: 0.8591768145561218, rewards: -43, count: 43
success rate:  0.39
epoch: 197429, loss: 0.47886401414871216, rewards: -56, count: 56
success rate:  0.37
epoch: 197439, loss: 1.043211579322815, rewards: -71, count: 71
success rate:  0.38
epoch: 197449, loss: 0.8529098629951477, rewards: -121, count: 112
success rate:  0.38
epoch: 197459, loss: 1.076446533203125, rewards: -57, count: 48
success rate:  0.4
epoch: 197469, loss: 0.7549585103988647, rewards: -62, count: 62
success rate:  0.39
epoch: 197479, loss: 0.9472728371620178, rewards: -280, count: 244
success rate:  0.4
epoch: 197489, loss: 0.7428974509239197, rewards: -182, count: 173
success rate:  0.41
epoch: 197499, loss: 0.8021920919418335, rewards: -78, count: 69
success rate:  0.4
epoch: 197509, loss: 0.7602431774139404, rewards: -477, count: 441
success rate:  0.4
epoch: 197519, loss: 1.4919967651367188, rewards: -153, count: 135
success rate:  0.35
epoch: 197529, loss: 0.8581448793411255,

success rate:  0.28
epoch: 198389, loss: 1.457705020904541, rewards: -318, count: 300
success rate:  0.31
epoch: 198399, loss: 1.099536418914795, rewards: -171, count: 171
success rate:  0.31
epoch: 198409, loss: 1.0310027599334717, rewards: -115, count: 115
success rate:  0.32
epoch: 198419, loss: 1.3522417545318604, rewards: -82, count: 73
success rate:  0.32
epoch: 198429, loss: 1.411118984222412, rewards: -154, count: 118
success rate:  0.33
epoch: 198439, loss: 1.0119622945785522, rewards: -401, count: 374
success rate:  0.37
epoch: 198449, loss: 0.42852839827537537, rewards: -74, count: 74
success rate:  0.39
epoch: 198459, loss: 0.8371725082397461, rewards: -350, count: 296
success rate:  0.36
epoch: 198469, loss: 0.9423803091049194, rewards: -412, count: 376
success rate:  0.35
epoch: 198479, loss: 0.9482819437980652, rewards: -71, count: 71
success rate:  0.35
epoch: 198489, loss: 0.9552966952323914, rewards: -158, count: 131
success rate:  0.32
epoch: 198499, loss: 0.14306084

success rate:  0.3
epoch: 199349, loss: 1.427699327468872, rewards: -109, count: 91
success rate:  0.35
epoch: 199359, loss: 0.9365514516830444, rewards: -52, count: 52
success rate:  0.36
epoch: 199369, loss: 1.248602032661438, rewards: -331, count: 295
success rate:  0.34
epoch: 199379, loss: 1.5159388780593872, rewards: -215, count: 197
success rate:  0.36
epoch: 199389, loss: 0.5022119283676147, rewards: -479, count: 443
success rate:  0.35
epoch: 199399, loss: 0.8627467155456543, rewards: -628, count: 538
success rate:  0.36
epoch: 199409, loss: 0.3831506669521332, rewards: -148, count: 139
success rate:  0.37
epoch: 199419, loss: 0.6820620894432068, rewards: -830, count: 758
success rate:  0.39
epoch: 199429, loss: 1.131239414215088, rewards: -195, count: 177
success rate:  0.39
epoch: 199439, loss: 0.8942668437957764, rewards: -408, count: 381
success rate:  0.38
epoch: 199449, loss: 0.9034056067466736, rewards: -60, count: 60
success rate:  0.35
epoch: 199459, loss: 1.173500418

In [34]:
def sample_action(self,state):
    probs = self.policy_net(state) # 4
    if np.random.uniform() < 0.01:
        action = np.random.randint(0,4)
        return action, torch.log(probs[action]+1e-8).detach()
    dist = Categorical(probs)
    action = dist.sample()
    log_prob = dist.log_prob(action)
    return action.item(),log_prob.detach()

# 替换方法
import types
agent.sample_action = types.MethodType(sample_action, agent)

In [16]:
import time
def visualize_agent(env, agent, num_episodes=5):
    """
    渲染显示智能体的行动
    """
    env = gym.make('CliffWalking-v0', render_mode='human')  # 创建可视化环境
    
    for episode in range(num_episodes):
        state_tuple = env.reset()
        state = state_tuple[0] if isinstance(state_tuple, tuple) else state_tuple
        total_reward = 0
        steps = 0
        done = False
        
        print(f"\nEpisode {episode + 1}")
        
        while not done:
            env.render()  # 渲染当前状态
            
            # 将状态转换为one-hot编码
            state_onehot = np.zeros(48)
            state_onehot[state] = 1
            
            # 使用训练好的策略选择动作
            with torch.no_grad():
                if np.random.random() < 0.0:
                    action = np.random.randint(0, 4)
                else:
                    state_tensor = torch.FloatTensor(state_onehot)
                    probs = agent.policy_net(state_tensor)
                    action = probs.argmax().item()  # 使用最可能的动作
            
            # 执行动作
            step_result = env.step(action)
            if len(step_result) == 4:
                next_state, reward, done, _ = step_result
            else:
                next_state, reward, terminated, truncated, _ = step_result
                done = terminated or truncated
            
            total_reward += reward
            steps += 1
            state = next_state
            
            # 添加小延迟使动作更容易观察
            time.sleep(0.5)
        
        print(f"Episode finished after {steps} steps. Total reward: {total_reward}")
    
    env.close()

# 在主程序最后添加：
if __name__ == "__main__":    
    # 训练完成后显示智能体行动
    print("\nVisualizing trained agent behavior...")
    env = gym.make('CliffWalking-v0',render_mode='human')
    visualize_agent(env, agent)


Visualizing trained agent behavior...


2025-04-27 13:52:09.865 python[67964:170339742] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-27 13:52:09.865 python[67964:170339742] +[IMKInputSession subclass]: chose IMKInputSession_Modern



Episode 1
Episode finished after 17 steps. Total reward: -17

Episode 2


KeyboardInterrupt: 

In [ ]:
env.close()